In [1]:
import os
import re
import random
import pandas as pd
import numpy as np
import json
import unicodedata
from tqdm.auto import tqdm
from ast import literal_eval
from typing import List, Dict
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import chisquare

import faiss
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

from multiprocessing import Pool, cpu_count
from FlagEmbedding import BGEM3FlagModel


In [2]:
index = faiss.read_index("wikipedia_bge_m3.index")

In [3]:
print(f"총 벡터 개수: {index.ntotal}")

총 벡터 개수: 1165197


In [2]:
ds = load_dataset("lcw99/wikipedia-korean-20240501", split="train")
df = ds.to_pandas()

df['doc_id'] = df.index

df_clean = df[['doc_id', 'title', 'text']].copy()

df_clean = df_clean[df_clean['text'].str.strip().astype(bool)]

print(df_clean.head())

   doc_id  title                                               text
0       0  지미 카터  '''제임스 얼 “지미” 카터 주니어'''(, 1924년 10월 1일~)는 민주당 ...
1       1     수학  '''수학'''(, , '''math''')은 수, 양, 구조, 공간, 변화 등의 ...
2       2  수학 상수  '''수학'''에서 '''상수'''란 그 값이 변하지 않는 불변량으로, 변수의 반대...
3       3     문학  파일:Fragonard, The Reader.jpg|섬네일|250px|장오노레 프라...
4       4  나라 목록  스위스 제네바에 있는 국제 연합 회원국 및 비회원 GA 옵서버의 국기\n이 목록에 ...


In [3]:
model_name = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:

def chunk_text_tokenwise(
    text: str,
    title: str,
    doc_id: int,
    tokenizer,
    max_tokens: int = 512,
    overlap: int = 50,
    safety_margin: int = 8,
    min_tokens: int = 80,
) -> List[Dict]:
    """
    - chunk 대상: text
    - 최종 입력은 title + chunk_text 를 가정하므로,
      text chunk는 effective_max 토큰으로 제한
    - overlap은 토큰 단위로 보장
    """

    if not isinstance(text, str) or not text.strip():
        return []

    title = "" if title is None else str(title).strip()
    title_tok = tokenizer.encode(title, add_special_tokens=False) if title else []
    effective_max = max(32, max_tokens - len(title_tok) - safety_margin)  # 최소 32 보장

    # 문장/문단 단위 split
    # - \n+ 로 문단을 나누고, 마침표/물음표/느낌표 뒤 공백에서도 나눔
    units = re.split(r'(?<=[.!?])\s+|\n+', text)
    units = [u.strip() for u in units if u and u.strip()]

    chunks = []
    cur_tokens = []
    chunk_idx = 0

    def flush_chunk(tokens):
        nonlocal chunk_idx
        if not tokens:
            return
        chunk_text = tokenizer.decode(tokens, skip_special_tokens=True, clean_up_tokenization_spaces=True).strip()
        if not chunk_text:
            return
        chunks.append({
            "chunk_id": f"{doc_id}_{chunk_idx}",
            "doc_id": doc_id,
            "title": title,
            "text": chunk_text,
            "offset": chunk_idx,
            "token_len": len(tokens),
        })
        chunk_idx += 1

    for u in units:
        u_tokens = tokenizer.encode(u, add_special_tokens=False)
        if not u_tokens:
            continue

        # 유닛 자체가 effective_max보다 길면: 토큰 슬라이딩으로 쪼갬
        if len(u_tokens) > effective_max:
            # 현재 누적분 먼저 flush
            flush_chunk(cur_tokens)
            cur_tokens = []

            step = max(1, effective_max - overlap)
            for i in range(0, len(u_tokens), step):
                sub = u_tokens[i:i + effective_max]
                flush_chunk(sub)
            continue

        # 누적 시 초과하면 flush + overlap 토큰 유지
        if len(cur_tokens) + len(u_tokens) > effective_max and cur_tokens:
            flush_chunk(cur_tokens)

            # 토큰 단위 overlap 유지
            keep = min(overlap, len(cur_tokens))
            cur_tokens = cur_tokens[-keep:]  # 다음 chunk 시작에 붙일 overlap

        cur_tokens.extend(u_tokens)

    # 마지막 flush
    flush_chunk(cur_tokens)

    # 너무 짧은 chunk 후처리(옵션): min_tokens 미만이면 앞 chunk에 합치기
    if min_tokens and len(chunks) >= 2:
        merged = []
        for c in chunks:
            if merged and c["token_len"] < min_tokens:
                # 이전 chunk에 합치기
                prev = merged[-1]
                prev_text = prev["text"] + " " + c["text"]
                prev_tokens = tokenizer.encode(prev_text, add_special_tokens=False)

                # 다시 effective_max 맞추기(넘치면 그냥 유지하거나, 뒤를 자르는 정책 선택)
                if len(prev_tokens) <= effective_max:
                    prev["text"] = prev_text
                    prev["token_len"] = len(prev_tokens)
                else:
                    # 넘치면: 합치지 않고 그냥 독립 chunk로 둠(보수적)
                    merged.append(c)
            else:
                merged.append(c)
        chunks = merged

    return chunks

In [5]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from joblib import Parallel, delayed, cpu_count
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

def process_batch(batch_df):
    batch_results = []
    # 전역 tokenizer 사용
    for text, title, doc_id in zip(batch_df['text'], batch_df['title'], batch_df['doc_id']):
        # 기존 로직 그대로 수행
        chunks = chunk_text_tokenwise(
            text=text, 
            title=title, 
            doc_id=doc_id, 
            tokenizer=tokenizer, 
            max_tokens=512, 
            overlap=50
        )
        batch_results.extend(chunks)
    return batch_results

BATCH_SIZE = 1000  # 한 번에 1000개씩 처리 (적절한 크기)
num_cores = cpu_count()


num_batches = len(df_clean) // BATCH_SIZE + 1
batches = np.array_split(df_clean, num_batches)

print(f"CPU {num_cores}개 사용 / 총 {len(batches)}개 배치로 묶어서 처리 시작...")

results = Parallel(n_jobs=-1, backend='loky')(
    delayed(process_batch)(batch)
    for batch in tqdm(batches)
)

all_chunk_records = [chunk for sublist in results for chunk in sublist]
df_chunks = pd.DataFrame(all_chunk_records)

print(f"청킹 완료! 총 청크 개수: {len(df_chunks)}")
print(df_chunks.head())

CPU 5개 사용 / 총 516개 배치로 묶어서 처리 시작...


/data/ephemeral/pro-nlp-generationfornlp-nlp-13/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/516 [00:00<?, ?it/s]

/data/ephemeral/pro-nlp-generationfornlp-nlp-13/.venv/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


청킹 완료! 총 청크 개수: 1165197
  chunk_id  doc_id  title                                               text  \
0      0_0       0  지미 카터  '''제임스 얼 “지미” 카터 주니어'''(, 1924년 10월 1일~)는 민주당 ...   
1      0_1       0  지미 카터  주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다....   
2      0_2       0  지미 카터  6,000명을 감축하는 데 그쳤다. 또한 박정희 정권의 인권 문제 등과의 논란으로 ...   
3      0_3       0  지미 카터  과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 ...   
4      0_4       0  지미 카터  자들 같은 인권유린범죄자를 재판소로 회부하여 국제적인 처벌을 받게 하는 등 인권 신...   

   offset  token_len  
0       0        457  
1       1        485  
2       2        433  
3       3        483  
4       4        476  


In [12]:
print(f"현재 텍스트 데이터 개수: {len(df_chunks)}")


target_columns = ['doc_id', 'title', 'text', 'chunk_id']

available_cols = [c for c in target_columns if c in df_chunks.columns]

print("메타데이터 저장 중...")
df_chunks[available_cols].to_parquet("wikipedia_chunks_meta.parquet", index=False)

print("\n🎉 진짜_진짜_최종_완료.parquet")
print("이제 3가지 파일이 모두 준비되었습니다.")

현재 텍스트 데이터 개수: 1165197
메타데이터 저장 중...

🎉 진짜_진짜_최종_완료.parquet
이제 3가지 파일이 모두 준비되었습니다.


In [14]:
### 검색
model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)
index = faiss.read_index("wikipedia_bge_m3.index")
df = pd.read_parquet("wikipedia_chunks_meta.parquet")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
def search(query, k=5):
    """
    query: 질문 텍스트
    k: 상위 몇 개를 보여줄지
    Return: [[score, title, text], [score, title, text], ...] 형태의 리스트
    """
    query_vecs = model.encode(
        [query], 
        batch_size=1, 
        max_length=2048 # 요청하신 길이 유지
    )['dense_vecs']

    query_embedding = query_vecs.astype("float32")
    faiss.normalize_L2(query_embedding)
    
    D, I = index.search(query_embedding, k)
    
    result_list = []    
    for rank, idx in enumerate(I[0]):
        # 인덱스 유효성 검사
        if idx < 0 or idx >= len(df):
            continue
            
        doc = df.iloc[idx]
        score = float(D[0][rank]) # numpy float를 일반 float로 변환
        
        # 원하시는 구조: [점수, 제목, 본문]
        result_list.append([score, doc['title'], doc['text']])
        
    return result_list

In [25]:
result1 = search("대한민국의 수도는 어디인가요?")

pre tokenize: 100%|██████████| 1/1 [00:00<00:00, 2146.52it/s]


Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 59.86it/s]


In [30]:
result1[0][-1]

'대한민국의 영토: 100,032km2 - 세계 25위 * 대한민국의 국립공원 ===대한민국의 지형=== * 대한민국의 폭포 목록 * 한국의 강 * 한국의 산 목록 * 한국의 섬 목록 * 한국의 호수 목록 ===대한민국의 지역=== * 한국의 지방 구분 ===대한민국의 행정 구역=== * 한국의 행정 구역 ===대한민국의 도시=== * 대한민국의 설치순 도시 목록 ** 대한민국의 수도: 서울특별시 ===대한민국의 인구=== * 대한민국의 수도: 서울특별시 * 대한민국의 선거 * 대한민국의 정당 * 대한민국 정부 ===행정부=== * 국가원수 겸 정부수반: 윤석열 대통령 * 국무총리: 한덕수 * 국무회의 ===입법부=== * 국회의장: 김진표 ===사법부=== * 대법원장: 김명수 * 헌법재판소장: 유남석 ===대한민국의 대외 관계=== * 대한민국의 재외공관 목록 * 대한민국 주재 외국공관 목록 * 남북 관계 ====대한민국이 가입한 국제 기구==== 대한민국은 다음 국제 기구들의 회원국이다: * 경제협력개발기구 * 국제 개발 협회 * 국제결제은행 * 국제 금융 공사 * 국제 노동 기구 * 국제노동조합연합 * 국제 농업 개발 기금 * 국제 민간 항공 기구 * 국제부흥개발은행 * 국제 상업 회의소 * 국제 수로 기구 * 국제 에너지 기구 * 국제 올림픽 위원회 * 국제 원자력 기구 * 국제 의회 연맹 * 국제이주기구 * 국제 적십자사·적신월사 연맹 * 국제 적십자·적신월 운동 * 국제전기통신연합 * 국제 통화 기금 * 국제 표준화 기구 * 국제해사기구 * 국제형사경찰기구 * 국제형사재판소 * 남아시아 지역 협력 연합 (참관국) * 다자간 투자 보증 기구 * 동남아시아 국가 연합 (아세안+3) * 동아시아 정상회의 * 라틴 아메리카 통합 연합 * 만국 우편 연합 * 미주개발은행 * 미주 기구 (참관국) * 상설중재재판소 * 세계 관광 기구 * 세계 관세 기구 * 세계 기상 기구 * 세계노동연합'

### 예시 확인


In [31]:
ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
dataset = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 63.6+ KB


In [32]:
# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             2031 non-null   object
 1   paragraph      2031 non-null   object
 2   question       2031 non-null   object
 3   choices        2031 non-null   object
 4   answer         2031 non-null   int64 
 5   question_plus  0 non-null      object
dtypes: int64(1), object(5)
memory usage: 95.3+ KB


In [35]:
df["choices_len"] = df["choices"].apply(len)

In [41]:
df4 = df[df['choices_len'] == 4]

In [43]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 792 entries, 0 to 791
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             792 non-null    object
 1   paragraph      792 non-null    object
 2   question       792 non-null    object
 3   choices        792 non-null    object
 4   answer         792 non-null    int64 
 5   question_plus  0 non-null      object
 6   choices_len    792 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 49.5+ KB


In [50]:

def run_retrieval_and_merge(
    question_df: pd.DataFrame,  # 원본 데이터셋 (df4)
    meta_df: pd.DataFrame,      # 검색 대상 문서 (df)
    index: faiss.Index,         # FAISS 인덱스
    model,                      # 임베딩 모델
    n: int = 10,                # 샘플링 개수 (전체를 하려면 len(df4) 넣으면 됨)
    k: int = 5,                 # Top-K (몇 배로 늘릴지)
    seed: int = 42              # 랜덤 시드
):
    """
    질문 데이터셋을 검색하여, 원본 컬럼 + 검색 결과를 합친 '확장된 DataFrame'을 반환합니다.
    행 개수는 n * k 개가 됩니다.
    """
    
    if index.ntotal != len(meta_df):
        print(f"[주의] 인덱스({index.ntotal})와 문서({len(meta_df)}) 개수 불일치")
    

    real_n = min(n, len(question_df))
    samples = question_df.sample(n=real_n, random_state=seed).copy()
    
    print(f"{real_n}개의 질문에 대해 Top-{k} 검색을 수행합니다.")
    print(f"   예상되는 결과 행의 개수: {real_n * k}개\n")
    
    merged_results = []

    for i, (idx, row) in enumerate(tqdm(samples.iterrows(), total=len(samples), desc="Processing")):
        
        # 쿼리 생성
        qid = row['id']
        query = f"{row['paragraph']} \n\n {row['question']}"
        
        # --- 검색 로직 ---
        q_vec = model.encode([query], batch_size=1, max_length=1024)['dense_vecs']
        q_vec = q_vec.astype("float32")
        faiss.normalize_L2(q_vec)
        D, I = index.search(q_vec, k)
        # ----------------
        
        # Top-K만큼 반복하며 원본 데이터 + 검색 데이터 결합
        for rank, doc_idx in enumerate(I[0]):
            if doc_idx < 0 or doc_idx >= len(meta_df):
                continue
            
            # 검색된 문서 가져오기
            retrieved_doc = meta_df.iloc[doc_idx]
            similarity_score = float(D[0][rank])
            
            # 이렇게 하면 id, paragraph, question, choices, answer 등이 다 들어감
            combined_row = row.to_dict()
            
            # 검색 결과 데이터 추가 (컬럼명 구분)
            combined_row['ctx_rank'] = rank + 1                # 순위
            combined_row['ctx_score'] = similarity_score       # 유사도 점수
            combined_row['ctx_title'] = retrieved_doc['title'] # 검색된 문서 제목
            combined_row['ctx_text'] = retrieved_doc['text']   # 검색된 문서 내용
            combined_row['ctx_id'] = retrieved_doc['doc_id']   # (있다면) 문서 ID
            
            merged_results.append(combined_row)

    # 4. DataFrame 생성
    final_df = pd.DataFrame(merged_results)
    
    print(f"\n완료! 총 {len(final_df)}개의 행이 생성되었습니다.")
    return final_df

In [68]:
expanded_df = run_retrieval_and_merge(
    question_df=df4,
    meta_df=df,
    index=index,
    model=model,
    n=3,   
    k=3   
)

# 결과 확인
print("="*50)
print(f"생성된 DataFrame 크기: {expanded_df.shape}")
print("="*50)

3개의 질문에 대해 Top-3 검색을 수행합니다.
   예상되는 결과 행의 개수: 9개



Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 58.41it/s]



완료! 총 9개의 행이 생성되었습니다.
생성된 DataFrame 크기: (9, 12)


In [64]:
expanded_df.iloc[9,:]

id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   generation-for-nlp-1160
paragraph        다음의 정보를 바탕으로 답을 고르시오. 아래 두 저자는 모두 프랑스 혁명에 대해 이야기하고 있다. 최고의 시절이자 최악의 시절, 지혜의 시대이자 어리석음의 시대였다. 믿음의 세기이자 의심의 세기였으며, 빛의 계절이자 어둠의 계절이었다. 희망의 봄이자 절망의 겨울이었다. 우리 앞에는 모든 것이 있었지만 한편으로 아무것도 없었다. 우리는 모두 천국으로 향해 가고자 했지만 우리는 엉뚱한 방향으로 걸었다. 말하자면, 지금과 너무 흡사하게, 그 시절 목청 큰 권위자들 역시 좋든 나쁘든 간에 오직 극단적인 비교로만 그 시대를 규정하려고 했다. 영국의 왕좌에는 턱이 큰 왕과 못생긴 왕비가 앉아 있었고, 프랑스의 왕좌에는 턱이 큰 왕과 아름다운 왕비가 앉아 있었다. 두 나라 모두 분명하게… 영원히 안정되었다는 사실이 분명해 졌다. - 찰스 디킨스, 두 도시 이야기 그 새벽에 살아 있음이 축복이었지만, 젊다는 건 바로 천국이었네. —서곡, 윌리엄 워즈워스
question      

In [70]:
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

sentences_1 = ["엘레노어 루즈벨트 연설에서 시민의 자유를 위협한 것은?"]
sentences_2 = [
    "프랭클린 루즈벨트의 네 가지 자유 연설은 1941년에 발표되었습니다.", # 문서 1 (남편)
    "엘레노어 루즈벨트는 대공황 시기 시민의 자유가 위축됨을 경고했습니다." # 문서 2 (부인 - 정답)
]

# 1. ColBERT 벡터 추출 (return_colbert_vecs=True 필수)
out1 = model.encode(sentences_1, return_colbert_vecs=True)
out2 = model.encode(sentences_2, return_colbert_vecs=True)

# 2. 점수 계산 (colbert_score 함수 사용)
# 질문의 각 토큰이 문서의 토큰 중 가장 유사한 것과 매칭되어 합산됨
score1 = model.colbert_score(out1['colbert_vecs'][0], out2['colbert_vecs'][0])
score2 = model.colbert_score(out1['colbert_vecs'][0], out2['colbert_vecs'][1])

print(f"문서 1 점수: {score1:.4f}")
print(f"문서 2 점수: {score2:.4f}")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


문서 1 점수: 0.5454
문서 2 점수: 0.7834


## Sparse

In [6]:
import gc

In [7]:
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

text_list = df_chunks['text'].tolist()
batch_size = 24        # GPU 메모리
save_interval = 10000  # ★ 1만 개마다 저장 (램 폭발 방지)
file_prefix = "wiki_sparse_part_"

buffer = []
part_idx = 0

print("Sparse 벡터 추출 및 분할 저장 시작...")

for i in tqdm(range(0, len(text_list), batch_size)):
    batch_texts = text_list[i : i + batch_size]
    
    output = model.encode(batch_texts, 
                          return_dense=False, 
                          return_sparse=True, 
                          return_colbert_vecs=False)
    
    buffer.extend(output['lexical_weights'])
    
    # 일정량 쌓이면 저장하고 메모리 비우기
    if len(buffer) >= save_interval:
        save_name = f"{file_prefix}{part_idx}.pkl"
        with open(save_name, 'wb') as f:
            pickle.dump(buffer, f)
        
        # print(f" -> {save_name} 저장 완료 (메모리 초기화)")
        buffer = [] 
        part_idx += 1
        gc.collect() # 메모리 청소

# 남은 데이터 저장
if buffer:
    save_name = f"{file_prefix}{part_idx}.pkl"
    with open(save_name, 'wb') as f:
        pickle.dump(buffer, f)
    print(f" -> {save_name} 최종 저장 완료")

print("🎉 모든 작업 완료! 폴더에 wiki_sparse_part_*.pkl 파일들이 있는지 확인하세요.")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Sparse 벡터 추출 및 분할 저장 시작...


  0%|          | 0/48550 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [10]:

model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

text_list = df_chunks['text'].tolist()
all_sparse_vecs = []
batch_size = 256 # GPU 메모리에 따라 조절

print("Sparse 벡터 추출 시작...")
for i in tqdm(range(0, len(text_list), batch_size)):
    batch_texts = text_list[i : i + batch_size]
    
    # dense는 계산하지 않고 sparse(lexical weights)만 가져옵니다.
    output = model.encode(batch_texts, 
                          return_dense=False, 
                          return_sparse=True, 
                          return_colbert_vecs=False)
    
    all_sparse_vecs.extend(output['lexical_weights'])

file_name = 'wikipedia_sparse_vecs.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(all_sparse_vecs, f)

print(f"저장 완료! 파일 크기를 확인하세요: {file_name}")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Sparse 벡터 추출 시작...


  0%|          | 0/4552 [00:00<?, ?it/s]

pre tokenize: 100%|██████████| 1/1 [00:00<00:00,  3.04it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.99s/it]


: 

In [ ]:
import pickle
import os
import gc # 가비지 컬렉터 (메모리 청소부)
from tqdm import tqdm
from FlagEmbedding import BGEM3FlagModel

# 1. 모델 로드
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

# 2. 전체 데이터 준비
text_list = df_chunks['text'].tolist()

# --- [설정 변경] ---
batch_size = 32
save_interval = 5000   # ★ 2만 -> 5천으로 줄임 (안전제일)
file_prefix = "wiki_sparse_part_"
# ------------------

# 3. 어디까지 했는지 확인 (Resume 로직)
existing_files = sorted([f for f in os.listdir() if f.startswith(file_prefix) and f.endswith('.pkl')])
processed_count = 0

if existing_files:
    # 이미 저장된 파일들에 담긴 데이터 개수 확인 (대략 계산)
    # 파일 이름이 part_0, part_1 순서라고 가정
    for f_name in existing_files:
        with open(f_name, 'rb') as f:
            data = pickle.load(f)
            processed_count += len(data)
    
    print(f"🔄 감지됨: 이미 {processed_count}개를 처리했습니다. 이어서 시작합니다!")
else:
    print("🚀 처음부터 시작합니다.")

# 4. 남은 부분만 돌리기
buffer = []
part_idx = len(existing_files) # 다음 번호부터 저장

# range(시작점, 끝, 배치)
for i in tqdm(range(processed_count, len(text_list), batch_size)):
    batch_texts = text_list[i : i + batch_size]
    
    # Sparse 벡터 추출
    output = model.encode(batch_texts, 
                          return_dense=False, 
                          return_sparse=True, 
                          return_colbert_vecs=False)
    
    buffer.extend(output['lexical_weights'])
    
    # 저장 주기마다 저장하고 비우기
    if len(buffer) >= save_interval:
        save_name = f"{file_prefix}{part_idx}.pkl"
        with open(save_name, 'wb') as f:
            pickle.dump(buffer, f)
            
        print(f" -> {save_name} 저장 완료 (메모리 청소)")
        buffer = [] 
        part_idx += 1
        gc.collect() # ★ 메모리 강제 수거

# 나머지 찌꺼기 저장
if buffer:
    save_name = f"{file_prefix}{part_idx}.pkl"
    with open(save_name, 'wb') as f:
        pickle.dump(buffer, f)
    print(f" -> {save_name} 최종 저장 완료")

print("🎉 전체 완료!")

In [72]:
# Sparse 벡터 로드
with open('wikipedia_sparse_vecs.pkl', 'rb') as f:
    sparse_vecs = pickle.load(f)

# 메타데이터 로드 (텍스트 확인용)
df_meta = pd.read_parquet('wikipedia_chunks_meta.parquet')

def compute_sparse_score(q_dict, d_dict):
    """두 딕셔너리 간의 가중치 곱 합산"""
    score = 0
    for token, weight in q_dict.items():
        if token in d_dict:
            score += weight * d_dict[token]
    return score

def get_sparse_top_k(query_text, k=5):
    # 질문에서 sparse 가중치 추출
    query_output = model.encode([query_text], return_dense=False, return_sparse=True)
    query_sparse = query_output['lexical_weights'][0]
    
    # 전체 sparse_vecs에 대해 점수 계산 (List Comprehension으로 속도 확보)
    all_scores = [compute_sparse_score(query_sparse, d_vec) for d_vec in sparse_vecs]
    
    # 상위 k개 인덱스 추출
    top_indices = np.argsort(all_scores)[::-1][:k]
    
    results = []
    for idx in top_indices:
        results.append({
            'index': idx,
            'score': all_scores[idx],
            'title': df_meta.iloc[idx]['title'],
            'text': df_meta.iloc[idx]['text']
        })
    
    return pd.DataFrame(results)

def verify_with_samples(df_samples, num_samples=3):
    samples = df_samples.sample(num_samples) # df4에서 랜덤 샘플 추출
    
    for i, row in samples.iterrows():
        print(f"\n{'='*50}")
        print(f"[질문]: {row['question']}")
        print(f"[실제 정답]: {row['answers']['text'][0] if 'answers' in row else 'N/A'}")
        print(f"{'='*50}")
        
        # Sparse 검색 수행
        search_results = get_sparse_top_k(row['question'], k=3)
        
        for j, res in search_results.iterrows():
            print(f"[{j+1}위] 점수: {res['score']:.4f} / 제목: {res['title']}")
            print(f"내용 요약: {res['text'][:150]}...")
            print("-" * 30)

# 실행
# df4가 로드되어 있다고 가정합니다.
verify_with_samples(df4, num_samples=3)

EOFError: Ran out of input

In [75]:
df_chunks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1165197 entries, 0 to 1165196
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   chunk_id   1165197 non-null  object
 1   doc_id     1165197 non-null  int64 
 2   title      1165197 non-null  object
 3   text       1165197 non-null  object
 4   offset     1165197 non-null  int64 
 5   token_len  1165197 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 53.3+ MB
